# Les centres d'intérêts des députés influencent-ils leur participation aux séances en commission permanente ?

## Introduction

Introduction ici: contexte général



présenter la motivation

<b> Stratégie d'idenfication </b>
 présenter la stratégie d'idenfication
 
 <b> Algorithmes </b>
 
 Présenter l'aspect plus algorithmique 
 
 
A demander : 

Récupérer le champ lexical


préférences des députés 



jouer sur le nombre de catégories, voir le threshold de déclenchement 

idée: 

• définir des catégories
•gradient de préférences pour le député
•gradient de thématiques pour la séance
•voir les paires de thématiques qui reviennent le plus souvent les unes à la suite des autres pour en déduire une proximité entre les différentes thématiques 
    - idée probabilité d'avoir un mot à la suite d'un autre 
    
• calculer une distance thématique/préférence, voir le pouvoir prédictif de cette distance


première étape pour les mots: créer une boucle, 1 ligne numero du député, une autre avec toutes ses interventions à la suite (sans traitement) 
filtrer pour les séances en hémicycle uniquement

à partir de ça voir les mots qui sortent le plus souvent pour chacun des députés




 


In [2]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Créer un profil de préférences pour chaque député

Première chose : créer les catégories thématiques (regrouper les champs lexicaux). Agreger toutes les interventions de tous les députés / prendre uniquement les séances en hémicycle et agréger pour tous les députés

A la fin, avoir des mots qui permettent de classifier une thématique de séance
faire du train/test pour voir si ça marche bien


## Attribuer une thématique aux séances en commission

## Voir comment la proximité thématique prédit la participation

## Bibliographie


<span>&#8226;</span> Benjamin Monnery & Maxime Le Bihan, 2018. "Can Public and Private Sanctions Discipline Politicians? Evidence from the French Parliament," EconomiX Working Papers 2018-21, University of Paris Nanterre, EconomiX.

<span>&#8226;</span> Wu, A. H. (2017). Gender stereotyping in academia: Evidence from economics job market rumors forum.
